In [1]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4.1-nano')
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D8DF2B32C0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D8DF553170>, root_client=<openai.OpenAI object at 0x000001D8DF1F3530>, root_async_client=<openai.AsyncOpenAI object at 0x000001D8DF7AC3E0>, model_name='gpt-4.1-nano', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [4]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

class Emailparser(BaseModel):
    sender_name : str = Field(description='이메일 송신자')# 보낸 사람
    title : str = Field(description='제목')# 제목 
    content : str = Field(description='내용')# 내용

parser = JsonOutputParser(pydantic_object=Emailparser)
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"sender_name": {"description": "이메일 송신자", "title": "Sender Name", "type": "string"}, "title": {"description": "제목", "title": "Title", "type": "string"}, "content": {"description": "내용", "title": "Content", "type": "string"}}, "required": ["sender_name", "title", "content"]}\n```'

In [7]:
prompt_temp = PromptTemplate(
    template='이메일을 읽고 정보를 추출하세요.\n{format_instructions}\n{email}',
    input_variables=['email'],
    partial_variables={'format_instructions': parser.get_format_instructions()}
)
prompt_temp

PromptTemplate(input_variables=['email'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"sender_name": {"description": "이메일 송신자", "title": "Sender Name", "type": "string"}, "title": {"description": "제목", "title": "Title", "type": "string"}, "content": {"description": "내용", "title": "Content", "type": "string"}}, "required": ["sender_name", "title", "content"]}\n```'}, template='이메일을 읽고 정보를 추출하세요.\n{format_instructions}\n{email}')

In [8]:
prompt = prompt_temp.format(email='보내는사람: 김철수\n제목: 금일자 회의록(9/8)\n회의내용: 오늘 분석팀과 진행한 회의록입니다.')
prompt

'이메일을 읽고 정보를 추출하세요.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"sender_name": {"description": "이메일 송신자", "title": "Sender Name", "type": "string"}, "title": {"description": "제목", "title": "Title", "type": "string"}, "content": {"description": "내용", "title": "Content", "type": "string"}}, "required": ["sender_name", "title", "content"]}\n```\n보내는사람: 김철수\n제목: 금일자 회의록(9/8)\n회의내용: 오늘 분석팀과 진행한 회의록입니다.'

In [9]:
response = llm.invoke(prompt)
response

AIMessage(content='{\n  "sender_name": "김철수",\n  "title": "금일자 회의록(9/8)",\n  "content": "오늘 분석팀과 진행한 회의록입니다."\n}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 262, 'total_tokens': 306, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_04d3664870', 'id': 'chatcmpl-CDMio0G6L17JwEa1ArahIbRD0GlJr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--0dffa90a-f937-463a-97cd-d7ec7cda7ffe-0', usage_metadata={'input_tokens': 262, 'output_tokens': 44, 'total_tokens': 306, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})